# GNN for Parity Games Experiments

## Raw Training Data Creation

**07 July 2022**

- Num games: 3000
- Graph size (N) range: [10,200]
- Relative outdegree $(\frac{d_i}{N})$ range: [0.01,0.5]
- Priority range: [0,N]

In [1]:
num_graphs = 3000
min_n = 10
max_n = 200
min_rod = 0.01
max_rod = 0.5

games_dir = 'games'
solutions_dir = 'solutions'
pgsolver_base = 'pgsolver' # Path to compiled base dir of https://github.com/tcsprojects/pgsolver 

In [2]:
#import game_generator as gg

#gg.create_games_and_solutions(num_graphs, min_n, max_n, min_rod, max_rod, games_dir, solutions_dir, pgsolver_base)

## Prepare Training

In [3]:
from torch_geometric.loader import DataLoader
from parity_game_dataset import ParityGameDataset
import math

data = ParityGameDataset('pg_data_20220708', 'games', 'solutions')

# Use first 70% of the data for training
split_index = math.floor(0.7 * len(data))
train_data = data[:split_index]
test_data = data[split_index:]

train_loader = DataLoader(train_data, batch_size=1, shuffle=True)
test_loader = DataLoader(test_data, batch_size=1, shuffle=False)



**Training Parameters**

- optimizer: Adam
- Learning rate: 0.001
- Loss: Cross Entropy
- Epochs: 1-100

## Prepare model

**Parameters**

- Iterations: 10
- ...

In [4]:
from importlib import reload
import parity_game_network as pn
#from parity_game_network import ParityGameNetwork
reload(pn)
model = pn.ParityGameNetwork(256, 256, 10)
#model = pn.ParityGameNetwork(128, 128, 5)

In [63]:
model

ParityGameNetwork(
  (core): GAT(3, 256, num_layers=10)
  (node_classifier): Sequential(
    (0): Linear(in_features=256, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=256, out_features=2, bias=True)
    (4): Softmax(dim=1)
  )
  (edge_classifier): Sequential(
    (0): Linear(in_features=512, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=256, out_features=2, bias=True)
    (4): Softmax(dim=1)
  )
)

# Run Training Loop

In [5]:
import wandb
wandb.init(project='gnn_parity_game_solver')
config = wandb.config
config.learning_rate = 0.001

wandb.watch(model, log='all')

wandb: Currently logged in as: alexdweinert. Use `wandb login --relogin` to force relogin


[]

In [6]:
import torch
import numpy as np
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion_nodes = torch.nn.CrossEntropyLoss()
criterion_edges = torch.nn.CrossEntropyLoss(weight=torch.tensor([0.1,0.9]))

def train():
    running_loss = 0.
    i = 0
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
        i += 1
        optimizer.zero_grad()  # Clear gradients.
        out_nodes, out_edges = model(data.x, data.edge_index)  # Perform a single forward pass.
        
        # Most edges do not belong to a winning strategy and thus the data is extemely imbalanced. The model will probably learn that predicting always "non-winning" for each edge 
        # yields reasonable performance. To avoid this, approximately as many non-winning strategy edges are sampled as there are winning edges.
        #edge_selection = (torch.rand(data.y_edges.shape[0]) > 0.7) | (data.y_edges == 1)
        loss = criterion_nodes(out_nodes, data.y_nodes) + criterion_edges(out_edges, data.y_edges) # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        
        running_loss += loss.item()
        if i % 50 == 0:
            last_loss = running_loss / 50 # loss per batch
            wandb.log({'loss': last_loss, 'variance': torch.var(out_nodes[:,1])})
            running_loss = 0.
            
def test(loader):
     model.eval()

     correct_nodes = 0
     correct_edges = 0

     for data in loader:  # Iterate in batches over the training/test dataset.
        out_nodes, out_edges = model(data.x, data.edge_index)  
        pred_nodes = out_nodes.argmax(dim=1)  # Use the class with highest probability.
        pred_edges = out_edges.argmax(dim=1)  # Use the class with highest probability.
        correct_nodes += (pred_nodes == data.y_nodes).sum() / len(pred_nodes)  # Check against ground-truth labels.
        correct_edges += (pred_edges == data.y_edges).sum() / len(pred_edges)
     return (correct_nodes / len(loader), correct_edges / len(loader))  # Derive ratio of correct predictions.

for epoch in range(1, 2):
    train()
    train_acc_nodes, train_acc_edges = test(train_loader)
    test_acc_nodes, test_acc_edges = test(test_loader)
    wandb.log({'Epoch': epoch, 'Acc_Acc_Nodes': train_acc_nodes, 'Test_Acc_Nodes': test_acc_nodes, 'Train_Acc_Edges': train_acc_edges, 'Test_Acc_Edges': test_acc_edges}) 
    print(f'Epoch: {epoch:03d}, Train Acc Nodes: {train_acc_nodes:.4f}, Test Acc Nodes: {test_acc_nodes:.4f}, Train Acc Edges: {train_acc_edges:.4f}, Test Acc Edges: {test_acc_edges:.4f}')

Epoch: 001, Train Acc Nodes: 0.9661, Test Acc Nodes: 0.9618, Train Acc Edges: 0.9124, Test Acc Edges: 0.9106


*Saving the following model:*
- Performance: Epoch: 001, Train Acc Nodes: 0.9832, Test Acc Nodes: 0.9824, Train Acc Edges: 0.9687, Test Acc Edges: 0.9685

In [9]:
print(model)

ParityGameNetwork(
  (core): GAT(3, 256, num_layers=10)
  (node_classifier): Sequential(
    (0): Linear(in_features=256, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=256, out_features=2, bias=True)
    (4): Softmax(dim=1)
  )
  (edge_classifier): Sequential(
    (0): Linear(in_features=512, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=256, out_features=2, bias=True)
    (4): Softmax(dim=1)
  )
)


In [10]:
torch.save(model.state_dict(), 'GAT_pg_solver_20220914.pth')

## Unparsing predicted strategies

In [7]:
it = iter(test_loader)
game = next(it)
game

DataBatch(x=[142, 3], edge_index=[2, 5176], y_nodes=[142], y_edges=[5176], batch=[142], ptr=[2])

In [9]:
it = iter(test_loader)
game = next(it)

def get_index_of_winning_edge(tensor):
    max_weight = None
    for i in range(len(tensor)):
        if tensor[i].item() == 1:
            return i
        
    return 0

print(f"parity {len(game.x)};")

winning_edge_index = [None] * len(game.x)

for edge_index in range(len(game.y_edges)):
    start_node = game.edge_index[0][edge_index]
    current_winning_edge_index = winning_edge_index[start_node]
    
    if current_winning_edge_index == None:
        winning_edge_index[start_node] = edge_index
        continue
    
    if game.y_edges[edge_index].item() > game.y_edges[current_winning_edge_index].item():
        winning_edge_index[start_node] = edge_index
        
for start_node in range(len(winning_edge_index)):
    owner = 0 if game.x[start_node][1].item() else 1
    next_node = game.edge_index[1][winning_edge_index[start_node]]
    print(f"{start_node} {owner} {next_node};")

parity 142;
0 1 2;
1 1 60;
2 1 2;
3 1 19;
4 0 15;
5 0 12;
6 0 50;
7 0 7;
8 0 8;
9 1 22;
10 1 22;
11 0 7;
12 0 26;
13 1 9;
14 1 19;
15 0 15;
16 0 18;
17 0 7;
18 0 7;
19 1 2;
20 0 17;
21 0 4;
22 1 19;
23 1 22;
24 1 2;
25 1 9;
26 0 26;
27 1 28;
28 1 2;
29 1 71;
30 1 2;
31 1 28;
32 1 2;
33 0 8;
34 0 26;
35 0 8;
36 0 48;
37 1 0;
38 0 7;
39 1 28;
40 0 8;
41 1 0;
42 1 24;
43 1 29;
44 0 7;
45 1 3;
46 0 8;
47 1 23;
48 0 8;
49 1 2;
50 0 35;
51 0 11;
52 1 52;
53 1 24;
54 0 8;
55 0 8;
56 0 7;
57 1 57;
58 0 7;
59 0 7;
60 1 2;
61 0 61;
62 0 62;
63 1 63;
64 1 37;
65 1 13;
66 1 0;
67 1 23;
68 0 7;
69 0 15;
70 0 38;
71 1 71;
72 1 2;
73 0 7;
74 1 3;
75 0 7;
76 0 8;
77 1 2;
78 1 78;
79 1 79;
80 1 53;
81 0 11;
82 1 52;
83 1 0;
84 1 0;
85 0 11;
86 1 86;
87 0 7;
88 0 4;
89 0 7;
90 1 0;
91 1 28;
92 1 0;
93 0 50;
94 0 7;
95 1 0;
96 1 0;
97 0 8;
98 1 2;
99 0 38;
100 1 0;
101 0 15;
102 0 11;
103 1 2;
104 0 104;
105 1 3;
106 0 70;
107 0 107;
108 0 8;
109 1 2;
110 0 17;
111 1 111;
112 1 31;
113 0 4;
114 1 2;
115 

## Model application example

In [11]:
it = iter(test_loader)
next(it)
example_game = next(it)
out_nodes, out_edges = model(example_game.x, example_game.edge_index) 
pred_nodes = out_nodes.argmax(dim=1)
pred_edges = out_edges.argmax(dim=1)

### Output 1: Winning reagions of players 0 and 1

**Predicted regions**

In [12]:
pred_nodes

tensor([1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
        0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0,
        0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0,
        1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0,
        1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0,
        0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
        1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0,
        1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1,
        1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,
        1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1,
        1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
        0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0,
        1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0,

**Actual winning regions (Calculated by pgsolver)**

In [13]:
example_game.y_nodes

tensor([1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
        0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0,
        0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0,
        1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0,
        1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0,
        0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
        1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0,
        1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1,
        1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
        1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1,
        1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
        0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0,
        1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0,

### Output 2: Winning strategies

A **1** means that the edge belongs to a winning strategy

**Predicted winning strategy**

In [14]:
example_game.edge_index[:,example_game.y_edges == 1]

tensor([[  0,   1,   2,  ..., 664, 665, 666],
        [ 40,   1,  26,  ..., 613, 605, 613]])

**Winning strategy from pgsolver**

In [78]:
example_game.edge_index[:,pred_edges == 1]

tensor([[  9,   9,  20, 101, 117, 117, 138, 138, 138, 138, 292, 292, 292],
        [ 12,  16,  16, 150,  51, 102,  71, 125, 153, 178, 250, 309, 334]])

In [72]:
max(pred_edges)

tensor(1)

In [43]:
example_game.y_edges.shape[0]

17852